In [1]:
%load_ext autoreload

%autoreload 2
import sys
import corner
import numpy as np
import matplotlib.pyplot as plt
import scipy
def binning(corner): return [[corner[i],corner[i+1]] for i in range(len(corner)-1)]
sys.path.append('/pbs/throng/lsst/users/cpayerne/LikelihoodsClusterAbundance/modules/')
import covariance as covar
import utils
import pandas as pd
import abundance as cl_count
import forecast
import emcee
from lnlikelihood import lnLikelihood
import pyccl as ccl
import edit, sys
import h5py, glob
Omega_c_true = 0.30711 - 0.048254
Omega_b_true = 0.048254
sigma8_true = .8288
Omegam_true = 0.30711
True_value = [Omega_c_true + Omega_b_true, sigma8_true]

/pbs/home/c/cpayerne/.conda/envs/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def load_pickle(filename, **kwargs):
    with open(filename, 'rb') as fin:
        return pickle.load(fin, **kwargs)

In [3]:
sys.path.append('/pbs/throng/lsst/users/cpayerne/LikelihoodsClusterAbundance/notebooks/Unbinned_likelihood_with_SSC')
import analysis

In [4]:
analysis_ = analysis.analysis['varym']

AttributeError: module 'analysis' has no attribute 'analysis'

In [ ]:
import matplotlib.pylab as pl
colors = plt.cm.get_cmap('jet', 9)
c = [colors(i) for i in range(9)]

In [ ]:
#import matplotlib as mpl
#c = np.array([14.3, 16])
#norm = mpl.colors.Normalize(vmin=c.min(), vmax=c.max())
#cmap = mpl.cm.ScalarMappable(norm=norm, cmap=mpl.cm.jet)
c#map.set_array([])

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(10,5), sharex='col')
#plt.style.use('dark_background')
#pssc/pnossc = LnoSSC/LSSC * contr
ls = '-'
for i in range(9):
        logmmin, logmmax = analysis_['logm_bins'][i]
        Om, y = np.load('Om_SSC_contribution_' + str(logmmin) + '_' + str(logmmax) + '.pkl', allow_pickle=True)
        ssc_contOm = np.exp([np.array(y)[i][2] for i in range(len(y))])
        lnLnoSSC = [np.array(y)[i][0] for i in range(len(y))]
        print(logmmax)
        def yOminterp(Omx):
            return np.interp(Omx, Om, ssc_contOm)
        
        lnf = np.log(yOminterp(Om+0.001))
        dlnf = lnf - np.log(yOminterp(Om-0.001))
        dlnfdOm = dlnf/0.002
        
        
        s8, y = np.load('s8_SSC_contribution_' + str(logmmin) + '_' + str(logmmax) + '.pkl', allow_pickle=True)
        ssc_conts8 = np.exp([np.array(y)[i][2] for i in range(len(y))])
        lnLnoSSCs8 = [np.array(y)[i][0] for i in range(len(y))]
        def ys8interp(s8x):
            return np.interp(s8x, s8, ssc_conts8)
        
        lnf = np.log(ys8interp(s8+0.001))
        dlnf = lnf - np.log(ys8interp(s8-0.001))
        dlnfds8 = dlnf/0.002

        #if i==8:
       # ls = '--'
    #else: ls = '-'
        #ax[0].plot(Om, dlnfdOm, color=c[i], ls=ls,)
        #ax[1].plot(s8, dlnfds8, color=c[i], ls=ls,)
        #c=cmap.to_rgba(logmmax)
        mask = (s8 > .58)*(s8 < 1)
        ax[1,0].plot(Om, np.log(ssc_contOm), color=c[i], ls=ls, lw=1)#,label = r'$\log_{10}(M_{\rm max}) = $'+f'${logmmax}$')
        ax[1,1].plot(s8[mask],np.log(ssc_conts8)[mask], color=c[i], ls=ls, lw=1)
        
        ax[0,0].plot(Om, lnLnoSSC-lnLnoSSC[0], color=c[i], ls='-', lw=1)
        
        ax[0,1].plot(s8[mask], np.array(lnLnoSSCs8)[mask] - np.array(lnLnoSSCs8)[mask][0], color=c[i], ls='-', lw=1)
        #ax[1].plot(s8, np.log(ssc_conts8)-np.log(ssc_conts8)[0], color=c[i], ls=ls,)

#ax[0].legend(fontsize=10, frameon=False)
ax[1,0].set_xlabel(r'$\Omega_m$', fontsize=20)
ax[1,1].set_xlabel(r'$\sigma_8$', fontsize=20)
#
ax[0,0].vlines(0.30711, -10000, 1000000, color='k', ls=(0.,(5,4)), zorder=0)
ax[1,0].vlines(0.30711, -100, 10, color='k', ls=(0.,(5,4)), zorder=0)
#ax[0].set_xlim(.2, .5)
#ax[0].set_ylim(5, 10.5)
ax[0,1].vlines(.8288, -10000, 1000000, color='k', ls=(0.,(5,4)), zorder=0)
ax[1,1].vlines(.8288, -10, 50, color='k', ls=(0.,(5,4)), zorder=0)

ax[1,1].set_xlim(.6, .95)
ax[0,0].set_xlim(.2, .5)


ax[0,0].set_ylim(-11000, 11000)
ax[1,0].set_ylim(4, 10)

ax[0,1].set_ylim(-1000, 130000)
ax[1,1].set_ylim(4, 12)
#fig.subplots_adjust(right=0.85)
#cbar_ax = fig.add_axes([0.87, 0.13, 0.01, .75])
#fig.colorbar(cmap, cax=cbar_ax, shrink=1,fraction=1 )
#cbar_ax.tick_params(labelsize=11)
#cbar_ax.tick_params(labelsize=17)
ax[0,0].set_ylabel(r'$\ln\mathcal{L}_{\rm ULC}$', fontsize=20)
ax[1,0].set_ylabel(r'$\ln(1 + f_{\rm SSC})$', fontsize=20)
#plt.tick_params(axis='both', which = 'major', labelsize= 13)
plt.savefig('unbinnedSSC_cs.png', bbox_inches='tight', dpi=300)